# Numpy Accumulate Example

In [1]:
import numpy as np

## `numpy.ufunc.accumulate`

From the documentaiton:

> `ufunc.accumulate(array, axis=0, dtype=None, out=None)`
>
> Accumulate the result of applying an operator to all elements of an array.
> 
> For a one-dimensional array, accumulate produces results equivalent to:

```
r = np.empty(len(A))
t = op.identity        # op = the ufunc being applied to A's  elements
for i in range(len(A)):
    t = op(t, A[i])
    r[i] = t
return r
```

You need to create a ufunc first. [This web page explains how](https://www.dmcdougall.co.uk/generic-accumulator-functions-using-numpy).

This is the best option in the case of a recurrent function that takes two scalars as input and outputs one scaler.

## Example - Adding elements using accumulate

In [2]:
import numpy as np

def test_add(x, data):
    return x + data

assert test_add(1, 2) == 3
assert test_add(2, 3) == 5

# Make a Numpy ufunc from my test_add function
test_add_ufunc = np.frompyfunc(test_add, 2, 1)

assert test_add_ufunc(1, 2) == 3
assert test_add_ufunc(2, 3) == 5
assert np.all(test_add_ufunc([1, 2], [2, 3]) == [3, 5])

data_sequence = np.array([1, 2, 3, 4])
f_out = test_add_ufunc.accumulate(data_sequence, dtype=object)
assert np.array_equal(f_out, [1, 3, 6, 10])

## Computing a recurrent equation with multiple inputs

See this Stackoverflow answer:
- https://stackoverflow.com/a/62736143/1609514

If we want to compute a recurrent equation that has more than one data input (and potentially more than one state variable too) this doesn't work.

If you try using `ufunc.accumulate` you get

```
ValueError: accumulate only supported for binary functions
```

## Option 1 - Using Python's builtin accumulate function

This solution isn't a vectorized calculation in numpy, but it does at least avoid a for loop.

In [3]:
from itertools import accumulate, chain


def t_next(t, data):
    Tm, tau = data  # Unpack more than one data input
    return Tm + (t - Tm)**tau

assert t_next(2, (0.38, 0)) == 1.38

t0 = 2  # Initial t
Tm_values = np.array([0.38, 0.88, 0.56, 0.67, 0.45, 0.98, 0.58, 0.72, 0.92, 0.82])
tau_values = np.linspace(0, 0.9, 10)

# Combine the input data into a 2D array
data_sequence = np.vstack([Tm_values, tau_values]).T
t_out = np.fromiter(accumulate(chain([t0], data_sequence), t_next), dtype=float)
print(t_out)
# [2.         1.38       1.81303299 1.60614649 1.65039964 1.52579703
#  1.71878078 1.66109554 1.67839293 1.72152195 1.73091672]

# Slightly more readable version possible in Python 3.8+
t_out = np.fromiter(accumulate(data_sequence, t_next, initial=t0), dtype=float)
print(t_out)
# [2.         1.38       1.81303299 1.60614649 1.65039964 1.52579703
#  1.71878078 1.66109554 1.67839293 1.72152195 1.73091672]

[2.         1.38       1.81303299 1.60614649 1.65039964 1.52579703
 1.71878078 1.66109554 1.67839293 1.72152195 1.73091672]
[2.         1.38       1.81303299 1.60614649 1.65039964 1.52579703
 1.71878078 1.66109554 1.67839293 1.72152195 1.73091672]


If anyone knows a way round that constraint I would be very interested.